In [1]:
import polars as pl
from op_datasets.processing.execute import reader, BatchInput
from op_datasets.logic.transforms.daily_address_summary import (
    daily_address_summary,
    CONDITIONS,
)
from op_coreutils.logger import structlog
from op_coreutils.bigquery.write import overwrite_partitions

from datetime import datetime
import os

os.environ["OPLABS_ENV"] = "PROD"


log = structlog.get_logger()

2024-10-17 22:20:10 [info     ] Loaded env var: OP_ANALYTICS_VAULT


In [2]:
inputbatch: BatchInput = None
for _inputbatch in reader(
    chain="op", block_spec="126071012:+500", source_spec="goldsky"
):
    inputbatch = _inputbatch
    break

dataframes = inputbatch.dataframes

2024-10-17 22:20:11 [info     ] Loaded 5 items into vault.    
2024-10-17 22:20:11 [info     ] Initialized Clickhouse client.
2024-10-17 22:20:12 [info     ] Query success: blocks in 1.33s 2000 rows
2024-10-17 22:20:13 [info     ] Query success: transactions in 2.52s 43972 rows
2024-10-17 22:20:14 [info     ] Query success: logs in 3.34s 187138 rows


In [3]:
dataframes

{'blocks': shape: (1_012, 24)
 ┌───────┬─────────┬──────────┬────────────┬───┬──────────┬───────────┬────────────┬────────────────┐
 │ chain ┆ network ┆ chain_id ┆ dt         ┆ … ┆ gas_used ┆ gas_limit ┆ extra_data ┆ transaction_co │
 │ ---   ┆ ---     ┆ ---      ┆ ---        ┆   ┆ ---      ┆ ---       ┆ ---        ┆ unt            │
 │ str   ┆ str     ┆ i32      ┆ str        ┆   ┆ i64      ┆ i64       ┆ str        ┆ ---            │
 │       ┆         ┆          ┆            ┆   ┆          ┆           ┆            ┆ i64            │
 ╞═══════╪═════════╪══════════╪════════════╪═══╪══════════╪═══════════╪════════════╪════════════════╡
 │ op    ┆ mainnet ┆ 10       ┆ 2024-09-30 ┆ … ┆ 1779886  ┆ 30000000  ┆ 0x         ┆ 14             │
 │ op    ┆ mainnet ┆ 10       ┆ 2024-09-30 ┆ … ┆ 4285527  ┆ 30000000  ┆ 0x         ┆ 27             │
 │ op    ┆ mainnet ┆ 10       ┆ 2024-09-30 ┆ … ┆ 6187833  ┆ 30000000  ┆ 0x         ┆ 30             │
 │ op    ┆ mainnet ┆ 10       ┆ 2024-09-30 ┆ … ┆ 516

In [4]:
# check if our blocks are in the range we want

log.info(f"Min block number: {dataframes['blocks']['number'].min()}")
log.info(f"Max block number: {dataframes['blocks']['number'].max()}")
log.info(
    f"Min block number: {datetime.fromtimestamp(dataframes['blocks']['timestamp'].min())}"
)
log.info(
    f"Max block number: {datetime.fromtimestamp(dataframes['blocks']['timestamp'].max())}"
)

2024-10-17 22:20:14 [info     ] Min block number: 126070000   
2024-10-17 22:20:14 [info     ] Max block number: 126071011   
2024-10-17 22:20:14 [info     ] Min block number: 2024-09-30 19:26:17
2024-10-17 22:20:14 [info     ] Max block number: 2024-09-30 19:59:59


In [5]:
result = daily_address_summary(
    dataframes["transactions"],
    conditions=CONDITIONS,
    address="from_address",
    chain_id="chain_id",
    chain="chain",
    dt="dt",
)

In [6]:
result.head()

from_address,chain_id,chain,dt,total_txs,total_txs_success,total_txs_fail,total_blocks,total_blocks_success,total_blocks_fail,min_block_number,max_block_number,block_interval_active,min_nonce,max_nonce,nonce_interval_active,total_l2_gas_used,total_l2_gas_used_success,total_l2_gas_used_fail,total_l1_gas_used,total_l1_gas_used_success,total_l1_gas_used_fail,total_gas_fees,total_gas_fees_success,total_gas_fees_fail,l2_contrib_gas_fees,l1_contrib_gas_fees,l1_blobgas_contrib_gas_fees,l1_l1gas_contrib_gas_fees,l2_contrib_gas_fees_base_fees,l2_contrib_gas_fees_priority_fees,avg_l2_gas_price_gwei,avg_l2_base_fee_gwei,avg_l2_priority_fee_gwei,avg_l1_gas_price_gwei,avg_l1_blob_base_fee_gwei,min_block_timestamp,max_block_timestamp,time_interval_active,unique_hours_active,perc_daily_hours_active,num_to_addresses,num_to_addresses_success,num_to_addresses_fail
str,i32,str,str,u32,u32,u32,u32,u32,u32,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,"decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,13]","decimal[*,13]","decimal[*,13]","decimal[*,13]","decimal[*,13]",datetime[μs],datetime[μs],duration[μs],u32,f64,u32,u32,u32
"""0x5fca00fdaad9e4b8c6ae8a4264c6…",10,"""op""","""2024-09-30""",9,9,0,9,9,0,126070221,126070998,778,7141,7149,9,543035,543035,0,29826,29826,0,0.0000020346019401950000,0.0000020346019401950000,0.0000000000000000000000,0.0000010801944224650000,0.0000009544075177300000,0.0000000302499169380000,0.0000000000000000000000,0.0000005371594224650000,0.0000005430350000000000,0.0019891801126,0.0009891801126,0.0010000000000,6.1209760809670,0.0000000010000,2024-09-30 23:33:39,2024-09-30 23:59:33,25m 54s,1,0.041667,1,1,0
"""0xf4911c3fc51ba098b42174175ead…",10,"""op""","""2024-09-30""",1,1,0,1,1,0,126070364,126070364,1,0,0,1,35212,35212,0,1600,1600,0,0.0000705176592191790000,0.0000705176592191790000,0.0000000000000000000000,0.0000704603652634240000,0.0000000572939557550000,0.0000000016227408000000,0.0000000000000000000000,0.0000000363652634240000,0.0000704240000000000000,2.0010327520000,0.0010327520000,2.0000000000000,6.8507216920000,0.0000000010000,2024-09-30 23:38:25,2024-09-30 23:38:25,0µs,1,0.041667,1,1,0
"""0x2973b942305df55b6fb1ed676a39…",10,"""op""","""2024-09-30""",1,1,0,1,1,0,126070098,126070098,1,243,243,1,40832,40832,0,1600,1600,0,0.0000001183029323210000,0.0000001183029323210000,0.0000000000000000000000,0.0000000728691138560000,0.0000000454338184650000,0.0000000016227408000000,0.0000000000000000000000,0.0000000320371138560000,0.0000000408320000000000,0.0017846080000,0.0007846080000,0.0010000000000,5.4325878090000,0.0000000010000,2024-09-30 23:29:33,2024-09-30 23:29:33,0µs,1,0.041667,1,1,0
"""0x9d960adcf4f919af4309a16c0e28…",10,"""op""","""2024-09-30""",1,1,0,1,1,0,126070822,126070822,1,16,16,1,21000,21000,0,1600,1600,0,0.0000001120831125150000,0.0000001120831125150000,0.0000000000000000000000,0.0000000458589390000000,0.0000000662241735150000,0.0000000016227408000000,0.0000000000000000000000,0.0000000458589390000000,0.0000000000000000000000,0.0021837590000,0.0021837590000,0.0000000000000,7.9185208310000,0.0000000010000,2024-09-30 23:53:41,2024-09-30 23:53:41,0µs,1,0.041667,1,1,0
"""0xa605857c215afb293770423f39bd…",10,"""op""","""2024-09-30""",1,1,0,1,1,0,126071010,126071010,1,453,453,1,53067,53067,0,1600,1600,0,0.0000001253853286940000,0.0000001253853286940000,0.0000000000000000000000,0.0000000642414243240000,0.0000000611439043700000,0.0000000016227408000000,0.0000000000000000000000,0.0000000589347243240000,0.0000000053067000000000,0.0012105720000,0.0011105720000,0.0001000000000,7.3110656530000,0.0000000010000,2024-09-30 23:59:57,2024-09-30 23:59:57,0µs,1,0.041667,1,1,0


In [7]:
overwrite_partitions(result, "temp", "daily_address_summary", expiration_days=999)

2024-10-17 22:20:14 [info     ] Writing 1 partitions to BQ [2024-09-30 00:00:00 ... 2024-09-30 00:00:00]
2024-10-17 22:20:14 [info     ] DRYRUN OVERWRITE PARTITION: Wrote 2.3Krows 283.7KB to BQ temp.daily_address_summary


In [8]:
# create test dfs
test_single_tx_df = dataframes["transactions"].filter(
    dataframes["transactions"]["hash"]
    == "0xc64c1471f38b0444d143011c0261a6c14d76a8b32115d39a3f3bf11af337eb55"
)
test_single_tx_df.write_parquet(
    "../../tests/op_datasets/test_daily_address_summary/test_daily_address_summary_single_tx.parquet"
)

test_multiple_txs_df = dataframes["transactions"].filter(
    dataframes["transactions"]["from_address"]
    == "0xd18a8be700b1fc5f7df8a3dbd772fbdecb16ee3a"
)

test_multiple_txs_df.write_parquet(
    "../../tests/op_datasets/test_daily_address_summary/test_daily_address_summary_multiple_txs.parquet"
)

In [9]:
test_result = daily_address_summary(
    test_single_tx_df,
    conditions=CONDITIONS,
    address="from_address",
    chain_id="chain_id",
    chain="chain",
    dt="dt",
)

In [10]:
test_result

from_address,chain_id,chain,dt,total_txs,total_txs_success,total_txs_fail,total_blocks,total_blocks_success,total_blocks_fail,min_block_number,max_block_number,block_interval_active,min_nonce,max_nonce,nonce_interval_active,total_l2_gas_used,total_l2_gas_used_success,total_l2_gas_used_fail,total_l1_gas_used,total_l1_gas_used_success,total_l1_gas_used_fail,total_gas_fees,total_gas_fees_success,total_gas_fees_fail,l2_contrib_gas_fees,l1_contrib_gas_fees,l1_blobgas_contrib_gas_fees,l1_l1gas_contrib_gas_fees,l2_contrib_gas_fees_base_fees,l2_contrib_gas_fees_priority_fees,avg_l2_gas_price_gwei,avg_l2_base_fee_gwei,avg_l2_priority_fee_gwei,avg_l1_gas_price_gwei,avg_l1_blob_base_fee_gwei,min_block_timestamp,max_block_timestamp,time_interval_active,unique_hours_active,perc_daily_hours_active,num_to_addresses,num_to_addresses_success,num_to_addresses_fail
str,i32,str,str,u32,u32,u32,u32,u32,u32,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,"decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,13]","decimal[*,13]","decimal[*,13]","decimal[*,13]","decimal[*,13]",datetime[μs],datetime[μs],duration[μs],u32,f64,u32,u32,u32
"""0xd2c52bfce3a40c8ec8cb79a64501…",10,"""op""","""2024-09-30""",1,0,1,1,0,1,126070912,126070912,1,5176,5176,1,119116,0,119116,2396,0,2396,0.0000031211542931570000,0.0000000000000000000000,0.0000031211542931570000,0.0000030281419376400000,0.0000000930123555170000,0.0000000024300543480000,0.0000000000000000000000,0.0000030281419376400000,0.0000000000000000000000,0.0254217900000,0.0254217900000,0.0000000000000,7.4238484020000,0.0000000010000,2024-09-30 23:56:41,2024-09-30 23:56:41,0µs,1,0.041667,1,0,1


In [11]:
test_df = dataframes["transactions"].filter(
    dataframes["transactions"]["hash"]
    == "0xc64c1471f38b0444d143011c0261a6c14d76a8b32115d39a3f3bf11af337eb55"
)
test_df

dt,chain,network,chain_id,block_timestamp,block_number,block_hash,hash,nonce,transaction_index,from_address,to_address,value_64,value_lossless,gas,gas_price,input,transaction_type,max_fee_per_gas,max_priority_fee_per_gas,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_status,receipt_effective_gas_price,receipt_l1_gas_price,receipt_l1_gas_used,receipt_l1_fee,receipt_l1_fee_scalar,receipt_l1_blob_base_fee,receipt_l1_blob_base_fee_scalar,receipt_l1_base_fee_scalar
str,str,str,i32,u32,i64,str,str,i64,i64,str,str,i64,str,i64,i64,str,i32,i64,i64,i64,i64,str,i32,i64,i64,i64,i64,f64,i64,i64,i64
"""2024-09-30""","""op""","""mainnet""",10,1727740601,126070912,"""0xb2533105b565d1e534999d4a4a8e…","""0xc64c1471f38b0444d143011c0261…",5176,4,"""0xd2c52bfce3a40c8ec8cb79a64501…","""0xcc78b2a3df6444a739badfb67410…",0,"""0""",1241000,25421790,"""0x0000000102040830725ac80454f1…",0,0,0,450678,119116,"""""",0,25421790,7423848402,2396,93012355517,null,1,1014213,5227


In [12]:
receipt_l1_gas_used = pl.select(test_df["receipt_l1_gas_used"]).item()
receipt_l1_base_fee_scalar = pl.select(test_df["receipt_l1_base_fee_scalar"]).item()
receipt_l1_gas_price = pl.select(test_df["receipt_l1_gas_price"]).item()

In [13]:
(receipt_l1_gas_used * receipt_l1_base_fee_scalar * receipt_l1_gas_price) / 1e18

0.09297547561102057

In [14]:
for i in list(test_multiple_txs_df["hash"]):
    print(i)

0x29ca6a42c2164ccddd912b227e3547685d34e27f63f0c29504b4420787a4c54f
0xdfe5c1ec2e815e37421e08cf838636e6d1c2ca3b069ca05a698c501e14920af5
0x4fe06c4733a8115872d016e976c60dcc8dba796beb583044acebc5048378a4aa
